## Importing libraries required

In [1]:
import numpy as np
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from tensorflow.keras.applications.inception_v3 import InceptionV3

## Creating deep learning model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='SparseCategoricalCrossentropy', metrics=['acc'])

## Creating directories for segregating the dataset

In [7]:
os.mkdir(f"{getcwd()}/tmp")
os.mkdir(f"{getcwd()}/tmp/training/")
os.mkdir(f"{getcwd()}/tmp/validation/")
os.mkdir(f"{getcwd()}/tmp/testing/")
os.mkdir(f"{getcwd()}/tmp/training/infected/")
os.mkdir(f"{getcwd()}/tmp/training/uninfected/")
os.mkdir(f"{getcwd()}/tmp/validation/infected/")
os.mkdir(f"{getcwd()}/tmp/validation/uninfected/")
os.mkdir(f"{getcwd()}/tmp/testing/infected/")
os.mkdir(f"{getcwd()}/tmp/testing/uninfected/")

## Segregating dataset into 3 parts for training, validation and testing

In [8]:
def split_data(SOURCE, TRAINING,VALIDATION, TESTING):
    files = os.listdir(SOURCE)
    files = random.sample(files,len(files))
    k = 0
    for file in files:
        k=k+1
        source = os.path.join(SOURCE,file)
        dest_tr = os.path.join(TRAINING,file)
        dest_tt = os.path.join(TESTING,file)
        dest_tv = os.path.join(VALIDATION,file)
        if os.path.getsize(source)!=0:
            if (k<=0.9*(len(files))):
                copyfile(source,dest_tr)
            elif (k<=0.95*(len(files))):
                copyfile(source,dest_tt)
            else:
                copyfile(source,dest_tv)
train_infec = f"{getcwd()}/tmp/training/infected/"
val_infec = f"{getcwd()}/tmp/validation/infected/"
test_infec = f"{getcwd()}/tmp/testing/infected/"
train_unin = f"{getcwd()}/tmp/training/uninfected/"
val_unin = f"{getcwd()}/tmp/validation/uninfected/"
test_unin = f"{getcwd()}/tmp/testing/uninfected/"
source_inf = f"{getcwd()}/cell_images/Parasitized"
source_uninf = f"{getcwd()}/cell_images/Uninfected"

In [9]:
split_data(source_inf,train_infec,val_infec,test_infec)

In [10]:
split_data(source_uninf,train_unin,val_unin,test_unin)

## Creating data generators for training and testing

In [2]:
TRAINING_DIR = f"{getcwd()}/tmp/training/"
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = f"{getcwd()}/tmp/validation/"
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

TEST_DIR = f"{getcwd()}/tmp/testing/"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(150, 150))


Found 24802 images belonging to 2 classes.
Found 1378 images belonging to 2 classes.
Found 1378 images belonging to 2 classes.


## Training the Model

In [59]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit(train_generator,
                              epochs=2,
                              verbose=1,
                              validation_data=validation_generator)


Epoch 1/2
776/776 [==============================] - 975s 1s/step - loss: 0.6801 - acc: 0.6694 - val_loss: 0.2833 - val_acc: 0.9158
Epoch 2/2
776/776 [==============================] - 1009s 1s/step - loss: 0.2980 - acc: 0.9021 - val_loss: 0.2064 - val_acc: 0.9463


## Saving the model

In [60]:
model.save('malaria_v2.h5')

## Evaluating the model on Test dataset

In [3]:
model = tf.keras.models.load_model('malaria_v2.h5')
loss, accuracy = model.evaluate(test_generator)

44/44 [==============================] - 26s 301ms/step - loss: 0.1892 - acc: 0.9499


In [5]:
print(f'Accuracy of model on test dataset is {accuracy}')

Accuracy of model on test dataset is 0.9499274492263794
